In [0]:
%python

#python pgm to find the frequencies of a given text/ string

def freq_text(text):
    freq={}
    for i in text:
        if i in freq:
              freq[i]+=1 #adding counter
        else:
            freq[i]=1 #initializing counter
    return freq  

text="abvdjhweh"
res=(freq_text(text))
print(res)
print(type(res))



In [0]:
%python


# For each customer_id, calculate the difference of total_amount between the current and previous order
from pyspark.sql.functions import col,lag
from pyspark.sql.window import Window


data = [
    ("c1", "2025-01-05", 150),
    ("c1", "2025-01-01", 100),
    ("c1", "2025-01-10", 120),
    ("c2", "2025-01-02", 200),
    ("c2", "2025-01-07", 180),
    ("c3", "2025-01-12", 90),
    ("c3", "2025-01-03", 50),
    ("c3", "2025-01-08", 70),
]
column=["customer_id","start_date","total_amt"]
df_data=spark.createDataFrame(data,column)
#display(df_data)
df_window=Window.partitionBy("customer_id").orderBy("start_date")
df_diff=(df_data.withColumn("diff_col",lag("total_amt").over(df_window))
         .withColumn("diff_col",col("total_amt")-col("diff_col")))
display(df_diff)





--sql
Table1   Table2
1  	 		2 
2	 		2
2	 		3
NULL	 NULL
         NULL
 
INNER JOIN, LEFT JOIN, RIGHT JOIN, CROSS JOIN output row counts..?
 inner join : 0+(2*2)=4 
 left : 4+2=6
 right : 4 + 3 =7
 cross : 20


--pyspark senario based
 >source - 100 gb (hadoop,S3) : present in s3
 >question is when u load this in parquet file in s3, how many files it will make.
spark code
line 1 - read the csv file from source
line 2 - write to output folder in parquet format

sol:
100 gb * 1024 mb / 128 mb for each = 800.parquet files
df.groupby("col1") --> groupby does default partitions :200


pyspark :

source
age - num(data type)
after 2 months, in the "age" field ->string values are coming , how will u handle it?

sol: while reafing the data , dont use 'infershema' use / create a schema naturally

like shown below





In [0]:
%python
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("order_date", StringType(), True),
    StructField("age", IntegerType(), True)   # force numeric type
])

df = spark.read.csv("/path/to/file.csv", schema=schema, header=True)


(tell which is job,stage,tasks) in given
Read csv file
show() 
groupBy()  
filter() 
repartition(100) 
join() 
filter()
write to output

sol :
Read CSV (lazy transformation)
        |
        v
show()  ---> Action → Job 1
        |
        v
Stage 1: Scan CSV partitions → Tasks = #input partitions
--------------------------------------------------------

groupBy() ---> Wide transformation → Shuffle → Stage 2
        |
        v
filter() ---> Narrow transformation → stays in Stage 2
        |
        v
repartition(100) ---> Wide transformation → Shuffle → Stage 3
        |
        v
join() ---> Wide transformation → Shuffle → Stage 4
        |
        v
filter() ---> Narrow transformation → stays in Stage 4
        |
        v
write() ---> Action → Job 2
        |
        v
Stage 5+: Shuffle + Output write → Tasks = #output partitions
